In [ ]:
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install mtranslate

In [ ]:
pip install googletrans

In [ ]:
nltk.download('stopwords')

In [ ]:
import pandas as pd

traindata = pd.read_csv('/kaggle/input/toxic-comment-classification/train.csv', encoding='ISO-8859-1')
print(traindata.head())


In [ ]:
testdata=pd.read_csv('/kaggle/input/toxic-comment-classification/test2.csv', encoding='ISO-8859-1')



In [ ]:
train=traindata[:round(0.7*len(traindata))]
#train_data=traindata['comment_text']
test=traindata[round(0.7*len(traindata)):]

In [ ]:
from mtranslate import translate

def translate_to_english(text):
    try:
        translated_text = translate(text, 'en', 'auto')
        return translated_text
    except Exception as e:
        return None


In [ ]:
def preprocessor(text):
    #Remove HTML tags
    text=BeautifulSoup(text,'html').get_text()
    text = re.sub('<[^>]', '', text)
    emoticons = re.findall('(?::|;|)(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    #letters=re.sub('[^a-zA-Z]',' ',raw_text)
    #words=text.lower().split()
    #remove stopwords
    words=text.lower().split()
    stop_words=set(stopwords.words('english'))
    words=[w for w in words if w not in stop_words]
    return words
    #return text

In [ ]:
len(traindata)

In [ ]:
len(testdata)

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
xtrain=[]
for t in train['comment_text']:
    t = ' '.join(preprocessor(t))
    #print(t)
    xtrain.append(t)

In [ ]:
xtest = []
for t in test['comment_text']:
    xtest.append(' '.join(preprocessor(t)))

  

In [ ]:
def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in text.split()]

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                            lowercase=False,
                            preprocessor=None)

In [ ]:
param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__tokenizer': [tokenizer_porter],
                   'clf__penalty': ['l2'],   # 'clf__penalty': ['l1', 'l2'],
                   'clf__C': [10.0]},
                  # {'vect__ngram_range': [(1, 1)],
                  #  'vect__tokenizer': [tokenizer, tokenizer_porter],
                  #  'vect__use_idf': [False],
                  #  'vect__norm': [None],
                  #  'clf__penalty': ['l2'],
                  #  'clf__C': [10.0]},
                  ]

In [ ]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain1=train['toxic']
ytrain1=list(ytrain1)
ytest1=test['toxic']
ytest1=list(ytest1)

In [ ]:
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf_toxic = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_toxic.fit(xtrain, ytrain1)

In [ ]:
print(gs_lr_tfidf_toxic.best_score_)
print(gs_lr_tfidf_toxic.best_params_)

In [ ]:
import sys
sys.setrecursionlimit(100000)
predytest_toxic=gs_lr_tfidf_toxic.predict(xtest)

In [ ]:
print(classification_report(ytest1, predytest_toxic,labels=[1,0]))

In [ ]:
ytrain2=train['severe_toxic']
ytrain2=list(ytrain2)
ytest2=test['severe_toxic']
ytest2=list(ytest2)

In [ ]:
gs_lr_tfidf_severe_toxic = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_severe_toxic.fit(xtrain, ytrain2)

In [ ]:
print(gs_lr_tfidf_severe_toxic.best_score_)
print(gs_lr_tfidf_severe_toxic.best_params_)
predytest_severe_toxic=gs_lr_tfidf_severe_toxic.predict(xtest)

In [ ]:
print(classification_report(ytest2, predytest_severe_toxic,labels=[1,0]))

In [ ]:
ytrain3=train['obscene']
ytrain3=list(ytrain3)
ytest3=test['obscene']
ytest3=list(ytest3)

In [ ]:
gs_lr_tfidf_obscene = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_obscene.fit(xtrain, ytrain3)

In [ ]:
print(gs_lr_tfidf_obscene.best_score_)
print(gs_lr_tfidf_obscene.best_params_)
predytest_obscene=gs_lr_tfidf_obscene.predict(xtest)

In [ ]:
print(classification_report(ytest3, predytest_obscene,labels=[1,0]))

In [ ]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain4=train['threat']
ytrain4=list(ytrain4)
ytest4=test['threat']
ytest4=list(ytest4)

In [ ]:
gs_lr_tfidf_threat = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_threat.fit(xtrain, ytrain4)

In [ ]:
print(gs_lr_tfidf_threat.best_score_)
print(gs_lr_tfidf_threat.best_params_)
predytest_threat=gs_lr_tfidf_threat.predict(xtest)
#predytestprob_threat=[]

In [ ]:
print(classification_report(ytest4,predytest_threat,labels=[1,0]))

In [ ]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain5=train['insult']
ytrain5=list(ytrain5)
ytest5=test['insult']
ytest5=list(ytest5)

In [ ]:
gs_lr_tfidf_insult = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_insult.fit(xtrain, ytrain5)

In [ ]:
print(gs_lr_tfidf_insult.best_score_)
print(gs_lr_tfidf_insult.best_params_)
predytest_insult=gs_lr_tfidf_insult.predict(xtest)
#predytestprob_insult=[]

In [ ]:
print(classification_report(ytest5,predytest_insult,labels=[1,0]))

In [ ]:
#ytrain1=traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
ytrain6=train['identity_hate']
ytrain6=list(ytrain6)
ytest6=test['identity_hate']
ytest6=list(ytest6)

In [ ]:
gs_lr_tfidf_identity_hate = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf_identity_hate.fit(xtrain, ytrain6)

In [ ]:
print(gs_lr_tfidf_identity_hate.best_score_)
print(gs_lr_tfidf_identity_hate.best_params_)
predytest_identity_hate=gs_lr_tfidf_identity_hate.predict(xtest)
#predytestprob_identity_hate=[]

In [ ]:
print(classification_report(ytest6, predytest_identity_hate,labels=[1,0]))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#traindata[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
a1=accuracy_score(ytest1,predytest_toxic)
a2=accuracy_score(ytest2,predytest_severe_toxic)
a3=accuracy_score(ytest3,predytest_obscene)
a4=accuracy_score(ytest4,predytest_threat)
a5=accuracy_score(ytest5,predytest_insult)
a6=accuracy_score(ytest6,predytest_identity_hate)
a=(a1+a2+a3+a4+a5+a6)/6
print("average accuracy:",a)

In [ ]:
from sklearn.metrics import hamming_loss

In [ ]:
predydataF={'1':predytest_toxic,'2':predytest_severe_toxic,'3':predytest_obscene,'4':predytest_threat,'5':predytest_insult,
            '6':predytest_identity_hate}
predydataF=pd.DataFrame(predydataF)
predydataF=predydataF.values

In [ ]:
testy_all=test[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
testy_all=testy_all.values

In [ ]:
xtraintf=xtest+xtrain

In [ ]:
xtesttf = []
for t in testdata['comment_text']:
    t = ' '.join(preprocessor(t))
    t = translate_to_english(t)
    if(t!=None):
        xtesttf.append(t)
    
    

In [ ]:
probs_ytest = gs_lr_tfidf_toxic.predict_proba(xtesttf)
probs_ytest_severe_toxic = gs_lr_tfidf_severe_toxic.predict_proba(xtesttf)
probs_ytest_obscene = gs_lr_tfidf_obscene.predict_proba(xtesttf)
probs_ytest_threat = gs_lr_tfidf_threat.predict_proba(xtesttf)
probs_ytest_insult = gs_lr_tfidf_insult.predict_proba(xtesttf)
probs_ytest_identity_hate = gs_lr_tfidf_identity_hate.predict_proba(xtesttf)

# Combine the probabilities into a single DataFrame
probs_df = pd.DataFrame({'id': testdata['id'],
                          'toxic': probs_ytest[:, 1],
                          'severe_toxic': probs_ytest_severe_toxic[:, 1],
                          'obscene': probs_ytest_obscene[:, 1],
                          'threat': probs_ytest_threat[:, 1],
                          'insult': probs_ytest_insult[:, 1],
                          'identity_hate': probs_ytest_identity_hate[:, 1]})

# Create the submission file
probs_df.to_csv('submission_probs.csv', index=False)